In [1]:
import glob
import yaml
import time
import numpy as np
import open3d as o3d
import pandas as pd
from pathlib import Path
from tqdm.auto import tqdm
from functools import partial
from tqdm.contrib import tzip
from sklearn.metrics import (accuracy_score, 
                             precision_score, 
                             recall_score)
from pointcloud_filters import (ROR_fn, SOR_fn, 
                                DROR_fn, DSOR_fn, 
                                LIOR_fn, DDIOR_fn, AGDOR_fn)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [8]:
# scan = np.fromfile("data_fog/1705396208_0820758.bin", dtype=np.float32).reshape(-1, 5)
# labels = np.fromfile("data_fog/1705396208_0820758.label", dtype=np.int32)
scan = np.fromfile("data_fog/1705396217_0796814.bin", dtype=np.float32).reshape(-1, 5)
labels = np.fromfile("data_fog/1705396217_0796814.label", dtype=np.int32)

mask = scan[:,2]<1.5
pcd = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(scan[mask,:3]))
colors = np.zeros((mask.sum(), 3))
colors[labels[mask]==0] = np.array([153.0, 157.0, 160.0])/255.0
colors[labels[mask]==1] = np.array([50.0, 97.0, 227.0])/255.0
pcd.colors = o3d.utility.Vector3dVector(colors)
o3d.visualization.draw_geometries([pcd])

In [3]:
inliers = ROR_fn(scan[:,:3], radius=0.3, min_neighbour=5)
ROR_preds = np.logical_not(inliers).astype(np.int32)

mask = scan[:,2]<1.5
pcd = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(scan[mask,:3]))
colors = np.zeros((mask.sum(), 3))
colors[ROR_preds[mask]==0] = np.array([153.0, 157.0, 160.0])/255.0
colors[ROR_preds[mask]==1] = np.array([255.0, 99.0, 71.0])/255.0
pcd.colors = o3d.utility.Vector3dVector(colors)
o3d.visualization.draw_geometries([pcd])
print(f"Precision: {precision_score(labels, ROR_preds)}, Recall: {recall_score(labels, ROR_preds)}")

Precision: 0.3732590529247911, Recall: 0.014587415632484215


In [5]:
inliers = SOR_fn(scan[:,:3], num_neighbour=5, std_mul=0.6)
SOR_preds = np.logical_not(inliers).astype(np.int32)
mask = scan[:,2]<1.5
pcd = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(scan[mask,:3]))
colors = np.zeros((mask.sum(), 3))
colors[SOR_preds[mask]==0] = np.array([153.0, 157.0, 160.0])/255.0
colors[SOR_preds[mask]==1] = np.array([255.0, 99.0, 71.0])/255.0
pcd.colors = o3d.utility.Vector3dVector(colors)
o3d.visualization.draw_geometries([pcd])
print(f"Precision: {precision_score(labels, SOR_preds)}, Recall: {recall_score(labels, SOR_preds)}")

Precision: 0.18241695303550973, Recall: 0.06934465490964511


In [6]:
inliers = DROR_fn(scan[:,:3], 
                  radius_multiplier=0.7, 
                  azimuth_angle_deg=2.8125, 
                  min_neighbour=5, 
                  min_search_radius=0, 
                  max_search_radius=10)
DROR_preds = np.logical_not(inliers).astype(np.int32)
mask = scan[:,2]<1.5
pcd = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(scan[mask,:3]))
colors = np.zeros((mask.sum(), 3))
colors[DROR_preds[mask]==0] = np.array([153.0, 157.0, 160.0])/255.0
colors[DROR_preds[mask]==1] = np.array([255.0, 99.0, 71.0])/255.0
pcd.colors = o3d.utility.Vector3dVector(colors)
o3d.visualization.draw_geometries([pcd])
print(f"Precision: {precision_score(labels, DROR_preds)}, Recall: {recall_score(labels, DROR_preds)}")

Precision: 0.665224032586558, Recall: 0.8533638145003266


In [7]:
inliers = DSOR_fn(scan[:,:3], 
                  num_neighbour=5, 
                  std_mul=0.15, 
                  range_mul=0.3, 
                  distance_upper_bound=np.inf)
DSOR_preds = np.logical_not(inliers).astype(np.int32)
mask = scan[:,2]<1.5
pcd = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(scan[mask,:3]))
colors = np.zeros((mask.sum(), 3))
colors[DSOR_preds[mask]==0] = np.array([153.0, 157.0, 160.0])/255.0
colors[DSOR_preds[mask]==1] = np.array([255.0, 99.0, 71.0])/255.0
pcd.colors = o3d.utility.Vector3dVector(colors)
o3d.visualization.draw_geometries([pcd])
print(f"Precision: {precision_score(labels, DSOR_preds)}, Recall: {recall_score(labels, DSOR_preds)}")

Precision: 0.6093429158110883, Recall: 0.9045286305247116


In [9]:
# Dense Fog (inside)
#         P      R
# ROR   0.373  0.014
# SOR   0.182  0.069
# DROR  0.665  0.853
# DSOR  0.609  0.904

In [10]:
# Dense Fog (outside)
#         P      R
# ROR   0.091  0.416
# SOR   0.074  0.799
# DROR  0.074  0.924
# DSOR  0.112  0.825